In [168]:
import os, sys, warnings
from pathlib import Path
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
#sys.path.insert(0, os.path.abspath(os.path.join(Path.home(), "rpa_libs")))

from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys

# Nossas libs
#import app
#from plugins.selenium.selenium import Selenium

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [185]:
# Exemplo de uso do arquivo de configuração
#app.redefine_config('caminho do seu arquivo .yaml')
#teste
import pandas as pd

In [170]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [171]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install().replace("THIRD_PARTY_NOTICES.", "")))

In [172]:
url = 'https://www.coingecko.com/pt'

In [173]:
driver.get(url)

In [174]:
table = driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[5]/table')

In [175]:
rows = table.find_elements(By.XPATH, './/tbody//tr')

In [176]:
table_data = []

In [177]:
for row in rows:
    cells = row.find_elements(By.XPATH, './/td')
    table_data.append([cell.text for cell in cells])

In [178]:
headers = [header.text for header in table.find_elements(By.XPATH, './/thead//th')]

In [179]:
headers = [item for item in headers if item]
table_data = [[cell for cell in row if cell.strip()] for row in table_data]


In [180]:
list(headers)

['#',
 'Moeda',
 'Preço',
 '1 h',
 '24 h',
 '7 d',
 'Volume em 24 h',
 'Capitalização de mercado',
 'Últimos 7 dias']

In [181]:
list(table_data)

[['1',
  'Bitcoin\nBTC',
  'Comprar',
  'US$ 63.467,90',
  '0.1%',
  '1.0%',
  '9.7%',
  'US$ 33.005.485.648',
  'US$ 1.253.998.284.409'],
 ['2',
  'Ethereum\nETH',
  'Comprar',
  'US$ 2.678,58',
  '0.2%',
  '4.3%',
  '17.0%',
  'US$ 20.546.987.329',
  'US$ 322.491.261.168'],
 ['3',
  'Tether\nUSDT',
  'Comprar',
  'US$ 1,00',
  '0.0%',
  '0.1%',
  '0.0%',
  'US$ 52.934.553.107',
  'US$ 119.279.197.018'],
 ['4',
  'BNB\nBNB',
  'Comprar',
  'US$ 612,11',
  '0.5%',
  '4.9%',
  '14.7%',
  'US$ 1.326.893.951',
  'US$ 89.346.401.496'],
 ['5',
  'Solana\nSOL',
  'Comprar',
  'US$ 144,13',
  '0.3%',
  '0.1%',
  '10.5%',
  'US$ 2.978.051.326',
  'US$ 67.528.575.036'],
 ['6',
  'USDC\nUSDC',
  'Comprar',
  'US$ 1,00',
  '0.1%',
  '0.1%',
  '0.0%',
  'US$ 7.547.493.461',
  'US$ 35.901.663.723'],
 ['7',
  'XRP\nXRP',
  'Comprar',
  'US$ 0,5877',
  '0.2%',
  '0.1%',
  '1.3%',
  'US$ 1.191.868.227',
  'US$ 33.181.003.307'],
 ['8',
  'Lido Staked Ether\nSTETH',
  'Comprar',
  'US$ 2.677,35',
  '0.1

In [182]:
df = pd.DataFrame(table_data, columns=headers)

In [183]:
df

,#,Moeda,Preço,1 h,24 h,7 d,Volume em 24 h,Capitalização de mercado,Últimos 7 dias
0,1,Bitcoin\nBTC,Comprar,"US$ 63.467,90",0.1%,1.0%,9.7%,US$ 33.005.485.648,US$ 1.253.998.284.409
1,2,Ethereum\nETH,Comprar,"US$ 2.678,58",0.2%,4.3%,17.0%,US$ 20.546.987.329,US$ 322.491.261.168
2,3,Tether\nUSDT,Comprar,"US$ 1,00",0.0%,0.1%,0.0%,US$ 52.934.553.107,US$ 119.279.197.018
3,4,BNB\nBNB,Comprar,"US$ 612,11",0.5%,4.9%,14.7%,US$ 1.326.893.951,US$ 89.346.401.496
4,5,Solana\nSOL,Comprar,"US$ 144,13",0.3%,0.1%,10.5%,US$ 2.978.051.326,US$ 67.528.575.036
...,...,...,...,...,...,...,...,...,...
95,96,Klaytn\nKLAY,"US$ 0,1366",0.1%,1.1%,7.9%,US$ 11.845.375,US$ 793.409.535,None
96,97,MultiversX\nEGLD,"US$ 28,67",0.5%,3.1%,16.7%,US$ 23.770.746,US$ 785.368.844,None
97,98,Notcoin\nNOT,"US$ 0,007634",0.5%,5.0%,6.4%,US$ 164.680.325,US$ 782.299.117,None
98,99,Tokenize Xchange\nTKX,"US$ 9,70",0.3%,1.7%,7.5%,US$ 7.919.035,US$ 776.363.951,None
